# Network Programming Cookbook

## Printing your machine's name and IPv4 addrees

In [7]:
import socket

In [28]:
socket.gethostname?

Docstring:
gethostname() -> string

Return the current host name.
Type:      builtin_function_or_method


In [8]:
host_name = socket.gethostname()

In [9]:
host_name

'macbook-air-marcin.home'

In [29]:
socket.gethostbyname?

Docstring:
gethostbyname(host) -> address

Return the IP address (a string of the form '255.255.255.255') for a host.
Type:      builtin_function_or_method


In [10]:
socket.gethostbyname(host_name)

'192.168.1.16'

## Retrieving a remote machine's IP address

In [11]:
remote_host = 'google.com'

In [12]:
socket.gethostbyname(remote_host)

'216.58.209.14'

## Converting an IPv4 address to different formats

When you deal with low-level network functions, sometimes, the usual string notation of IP addresses are not very useful. They need to be converted to the packed 32-bit binary formats.

In [14]:
ip_address = '127.0.0.1'

In [15]:
# string to packed
socket.inet_aton?

Docstring:
inet_aton(string) -> bytes giving packed 32-bit IP representation

Convert an IP address in string format (123.45.67.89) to the 32-bit packed
binary format used in low-level network functions.
Type:      builtin_function_or_method


In [18]:
packed = socket.inet_aton(ip_address)

In [19]:
packed

b'\x7f\x00\x00\x01'

In [23]:
# Sometimes it's useful to display it in a more friendly format

In [24]:
from binascii import hexlify

In [30]:
hexlify?

Docstring:
Hexadecimal representation of binary data.

  sep
    An optional single character or byte to separate hex bytes.
  bytes_per_sep
    How many bytes between separators.  Positive values count from the
    right, negative values count from the left.

The return value is a bytes object.  This function is also
available as "b2a_hex()".
Type:      builtin_function_or_method


In [25]:
hexlify(packed)

b'7f000001'

In [26]:
# 32-bit packed to string
socket.inet_ntoa?

Docstring:
inet_ntoa(packed_ip) -> ip_address_string

Convert an IP address from 32-bit packed binary format to string format
Type:      builtin_function_or_method


In [27]:
socket.inet_ntoa(packed)

'127.0.0.1'

## Finding a service name, given the port and protocol

In [31]:
socket.getservbyport?

Docstring:
getservbyport(port[, protocolname]) -> string

Return the service name from a port number and protocol name.
The optional protocol name, if given, should be 'tcp' or 'udp',
otherwise any protocol will match.
Type:      builtin_function_or_method


In [34]:
socket.getservbyport(80)

'http'

## Setting and getting the default socket timeout

In [35]:
s = socket.socket()

In [38]:
s.gettimeout?

Docstring:
gettimeout() -> timeout

Returns the timeout in seconds (float) associated with socket
operations. A timeout of None indicates that timeouts on socket
operations are disabled.
Type:      builtin_function_or_method


In [37]:
print(s.gettimeout())

None


In [39]:
s.settimeout(100)

## Modifying a socket's send/receive buffer sizes

In [40]:
send_buf_size = 4096
recv_buf_size = 4096

In [41]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [43]:
# Before
sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)

131072

In [45]:
# After 
sock.setsockopt(socket.SOL_TCP, socket.TCP_NODELAY, 1)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, send_buf_size)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, recv_buf_size)

In [46]:
sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)

4096

## Changing a socket to the blocking/non-blocking mode

By default TCP sockets are placed in a blocking mode.

In [47]:
s = socket.socket()

In [48]:
s.setblocking?

Docstring:
setblocking(flag)

Set the socket to blocking (flag is true) or non-blocking (false).
setblocking(True) is equivalent to settimeout(None);
setblocking(False) is equivalent to settimeout(0.0).
Type:      builtin_function_or_method


In [49]:
# blocking
s.setblocking(True)
# non-blocking
s.setblocking(False)

## Printing the current time from the internet time server

In [51]:
import ntplib
from time import ctime

In [53]:
ntp_client = ntplib.NTPClient()
r = ntp_client.request('pool.ntp.org')
print(ctime(r.tx_time))

Fri Apr 22 19:55:52 2022


## TCP client

The most minimal example possible

In [57]:
# %load tcp-client.py
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('localhost', 65433))
sock.sendall(b'hey')

## TCP server

In [58]:
# %load tcp-server.py
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('127.0.0.1', 65433))
sock.listen()

while True:
    conn, addr = sock.accept()
    data = conn.recv(1024)
    if not data:
        break
    else:
        print(f'Got {data} from {addr}')